# Week 3

During week 3 of the project, we worked on replacing the dino and obstacle blocks with bitmap images of a cartoon dinosaur and 3 different types of cactii to make it aesthetically pleasing. In order to do so, we first used the predefined `tft_drawBitmap()` function and successfully implemented the dino and cactii graphics in the game. However, we realized that the function is not optimized and takes a lot of CPU cycles to run. Thus, we digged into the header files and created our own optimized function for drawing bitmaps.

Next, we implemented the live score and high score functionality in the game.

---

## The Design

#### 1. The graphic images

In order to implement the graphics for the dino and cactii, we downloaded stock images of 3 different cactii, and 3 configurations of the dino - jump position, running position 1 and running position 2. 

In order from left to right: small cactus, large cactus and multiple cactii.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/smallCactus.png" style="width: 20px; height: 40px;" >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/largeCactus.png" style="width: 30px; height: 60px;" >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/multiCactus.png" style="width: 60px; height: 40px;" >
</div>

In order from left to right: dino jump, running position 1 and running position 2.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/dinoJump.png" style="width: 44px; height: 50px;" >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/dinoRun1.png" style="width: 44px; height: 50px;" >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/dinoRun2.png" style="width: 44px; height: 50px;" >
</div>

<br>

#### 2. Converting images to bitmaps

In order to display these images in the game, we needed to convert them in a form which a microcontroller could understand. This form is known as a `bitmap`. A bitmap is basically an array of pixels where each bit of the array signifies whether a pixel is dark or light. If the pixel is light, the bit is set to 1 and if the pixel is dark, the bit is set to 0. In order to conserve space, the pixels are represented in a hexdecimal format so that each element of the array represents a series of 8 pixels.

In order to convert these PNG images in the bitmap format, we used [LCD Image Converter](https://lcd-image-converter.riuson.com/en/about/). We followed the following steps for this conversion:
- Open the desired PNG image in the software.
- Click on `Options > Conversion`.
- Set the preset to `Monochrome` and click `Show Preview`.
- This will open up a new window which displays the 1D array of the desired 2D bitmap image.

#### 3. Creating the bitmap header

The above conversion was repeated for all six images and we stored the resultant arrays in a file called `BitMap.h` which can be found [here](https://parthssharma.github.io/ECE4760/FinalProject/Files/BitMap.h). Every single array is of the type `const unsigned char`. We used `const` because the bitmap will remain constant throught the execution of the program and will be stored in the flash memory.

#### 4. Implementing the predefined bitmap function

The provided `tft_gfx.h` library has a predefined function called `tft_drawBitmap()` which takes in the following parameters to draw the image:
- The x-coordinate of the image
- The y-coordinate of the image
- The pointer to the first element of the bitmap array
- The width of the image
- The height of the image
- The color of the image

We implemented the code as below.

We first included the bitmap library.

```c
#include "BitMap.h"
```

<br>

We then defined `RUNNER_FRAMES` constant value and a variable `runner` which help us in creating an illusion of the dino running.

```c
#define RUNNER_FRAMES 10
char runner = 0;
```

<br>

The next step was to replace all the `tft_fillRect()` functions in the animation protothread with the `tft_drawBitmap()` function to implement the dino cactii graphics.

_Note: To implement the cactii graphics, we had to use a switch case which draws the correct bitmap for the cactii based on the variable `obsType`._

```c
switch(obsType){
    case 0: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeZer, obstacle.w, obstacle.h, ILI9340_BLACK);
            break;
    case 1: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeOne, obstacle.w, obstacle.h, ILI9340_BLACK);
            break;
    case 2: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeTwo, obstacle.w, obstacle.h, ILI9340_BLACK);
            break;
}

//After updating the obstacle parameters

switch(obsType){
    case 0: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeZer, obstacle.w, obstacle.h, HARD_COLOR);
            break;
    case 1: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeOne, obstacle.w, obstacle.h, HARD_COLOR);
            break;
    case 2: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeTwo, obstacle.w, obstacle.h, HARD_COLOR);
            break;
}
```

<br>

After updating the player parameters, we had to draw the appropriate dino image. We used the following three conditions to draw the dino:
- If the dino is in the middle of a jump, then we draw the dino jump bitmap.
- If the `runner` variable is between 0 and `RUNNER_FRAMES` / 2, then draw the dino running position 1 bitmap.
- If the `runner` variable is between `RUNNER_FRAMES` / 2 and `RUNNER_FRAMES`, then draw the dino running position 2 bitmap.

Once the `runner` variable hits `RUNNER_FRAMES`, it will reset to 0.

```c
if(myPlayer.y > 0){
    tft_drawBitmap(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), dinoJumpUp, myPlayer.w, myPlayer.h, SOFT_COLOR);
}
else{
    if(runner > RUNNER_FRAMES / 2){
        tft_drawBitmap(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), dinoRunOne, myPlayer.w, myPlayer.h, SOFT_COLOR);
    }
    else{
        tft_drawBitmap(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), dinoRunTwo, myPlayer.w, myPlayer.h, SOFT_COLOR);
    }
}
runner = (runner + 1) % RUNNER_FRAMES;
```

<br>